In [1]:
# %%
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, TrainValidationSplit
from pyspark.sql.functions import col
import time

# %%
spark = SparkSession.builder \
    .appName("SECOP_HyperparameterTuning") \
    .master("spark://spark-master:7077") \
    .getOrCreate()

print("✓ Sesión Spark iniciada")

# base de datos
df = spark.read.parquet("/opt/spark-data/raw/secop_ml_ready.parquet")

df = df.withColumnRenamed("valor_del_contrato_num", "label") \
       .withColumnRenamed("features_pca", "features") \
       .filter(col("label").isNotNull())

train, test = df.randomSplit([0.8, 0.2], seed=42)

print(f"Train: {train.count():,}")
print(f"Test: {test.count():,}")

# Modelo base y evaluador
lr = LinearRegression(
    featuresCol="features",
    labelCol="label",
    maxIter=100
)

evaluator = RegressionEvaluator(
    labelCol="label",
    predictionCol="prediction",
    metricName="rmse"
)

print("✓ Modelo base y evaluador configurados")


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/12 02:45:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


✓ Sesión Spark iniciada


Train: 80,062
Test: 19,938
✓ Modelo base y evaluador configurados


26/02/12 02:45:28 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


### **Diseño del Grid de Hiperparámetros**

**¿Por qué usamos escala logarítmica para regParam (0.01, 0.1, 1.0) en lugar de lineal (0.33, 0.66, 1.0)?**

Usamos escala logarítmica porque la regularización afecta el modelo de manera exponencial y no lineal. 
Pequeños cambios en valores bajos de λ pueden generar cambios significativos en los coeficientes.

Explorar valores como 0.01, 0.1 y 1.0 permite cubrir diferentes órdenes de magnitud y detectar rápidamente
si el modelo necesita poca, moderada o alta regularización.

Una escala lineal podría concentrarse en una zona poco informativa y no capturar correctamente el impacto real
de la regularización.


In [2]:
grid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.01, 0.1, 1.0]) \
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0]) \
    .addGrid(lr.maxIter, [50, 100, 200]) \
    .build()

print(f"Combinaciones totales: {len(grid)}")

K = 3
print(f"Modelos totales a entrenar con K={K}: {len(grid) * K}")


Combinaciones totales: 27
Modelos totales a entrenar con K=3: 81


Combinaciones totales generadas: 27  
(3 valores de regParam × 3 de elasticNetParam × 3 de maxIter)

Si usamos K=3 folds:

Total modelos entrenados = 27 × 3 = 81 modelos


### **Implementar Grid Search + Cross-Validation**

Usamos K=3 en lugar de K=5 porque:

- Reduce el tiempo computacional significativamente.
- Ya estamos explorando múltiples combinaciones (27 en este caso).
- El dataset es grande (~100k registros), por lo que K=3 ofrece un buen balance entre robustez y costo computacional.
- K=5 aumentaría el tiempo de entrenamiento en un 66% adicional sin necesariamente mejorar sustancialmente la estimación.

En escenarios de tuning amplio, es recomendable comenzar con K pequeño y luego refinar con K mayor si es necesario.



In [4]:
# Configurar CrossValidator con K=3
cv_grid = CrossValidator(
    estimator=lr,
    estimatorParamMaps=grid,
    evaluator=evaluator,
    numFolds=3,
    seed=42
)

print("Entrenando Grid Search + Cross-Validation (K=3)...")

start_time = time.time()
cv_grid_model = cv_grid.fit(train)
grid_time = time.time() - start_time

print(f"✓ Grid Search + CV completado en {grid_time:.2f} segundos")

# %%
best_grid_model = cv_grid_model.bestModel

predictions_grid = best_grid_model.transform(test)
rmse_grid = evaluator.evaluate(predictions_grid)

print("\n=== MEJOR MODELO (Grid Search + CV) ===")
print(f"regParam:        {best_grid_model.getRegParam()}")
print(f"elasticNetParam: {best_grid_model.getElasticNetParam()}")
print(f"maxIter:         {best_grid_model.getMaxIter()}")
print(f"RMSE en Test:    ${rmse_grid:,.2f}")



Entrenando Grid Search + Cross-Validation (K=3)...
✓ Grid Search + CV completado en 18.63 segundos

=== MEJOR MODELO (Grid Search + CV) ===
regParam:        1.0
elasticNetParam: 1.0
maxIter:         50
RMSE en Test:    $2,342,402,953.27


### **Implementar Train-Validation Split**

En esta estrategia se divide el conjunto de entrenamiento en un único split (80% entrenamiento y 20% validación), en lugar de usar K folds.

Ventajas:
- Mucho más rápido que Cross-Validation.
- Cada combinación se entrena solo una vez.

Desventajas:
- La métrica depende de un único split.
- Puede ser menos estable si la partición no es representativa.

Usamos trainRatio = 0.8 porque:
- Es el estándar clásico (80/20).
- Mantiene suficiente información para entrenar.
- Permite validar sin desperdiciar demasiados datos.emasiados datos.


In [5]:
from pyspark.ml.tuning import TrainValidationSplit

tvs = TrainValidationSplit(
    estimator=lr,
    estimatorParamMaps=grid,
    evaluator=evaluator,
    trainRatio=0.8,
    seed=42
)

print("Entrenando con Train-Validation Split...")

start_time = time.time()
tvs_model = tvs.fit(train)
tvs_time = time.time() - start_time

print(f"✓ Train-Validation completado en {tvs_time:.2f} segundos")


Entrenando con Train-Validation Split...
✓ Train-Validation completado en 6.66 segundos


In [6]:
# Obtener Mejor Modelo y Evaluar en Test
best_tvs_model = tvs_model.bestModel

predictions_tvs = best_tvs_model.transform(test)
rmse_tvs = evaluator.evaluate(predictions_tvs)

print("\n=== MEJOR MODELO (Train-Validation Split) ===")
print(f"regParam:        {best_tvs_model.getRegParam()}")
print(f"elasticNetParam: {best_tvs_model.getElasticNetParam()}")
print(f"maxIter:         {best_tvs_model.getMaxIter()}")
print(f"RMSE en Test:    ${rmse_tvs:,.2f}")



=== MEJOR MODELO (Train-Validation Split) ===
regParam:        1.0
elasticNetParam: 1.0
maxIter:         50
RMSE en Test:    $2,342,402,953.27


### **Comparar ambas estrategias (rendimiento vs velocidad)**

In [7]:
print("COMPARACIÓN DE ESTRATEGIAS")
print("="*60)

print("Grid Search + CV:")
print(f"  - Tiempo: {grid_time:.2f}s")
print(f"  - RMSE Test: ${rmse_grid:,.2f}")
print(f"  - Hiperparámetros: λ={best_grid_model.getRegParam()}, "
      f"α={best_grid_model.getElasticNetParam()}, "
      f"maxIter={best_grid_model.getMaxIter()}")

print("\nTrain-Validation Split:")
print(f"  - Tiempo: {tvs_time:.2f}s")
print(f"  - RMSE Test: ${rmse_tvs:,.2f}")
print(f"  - Hiperparámetros: λ={best_tvs_model.getRegParam()}, "
      f"α={best_tvs_model.getElasticNetParam()}, "
      f"maxIter={best_tvs_model.getMaxIter()}")

print("\nDiferencia RMSE:", abs(rmse_grid - rmse_tvs))


COMPARACIÓN DE ESTRATEGIAS
Grid Search + CV:
  - Tiempo: 18.63s
  - RMSE Test: $2,342,402,953.27
  - Hiperparámetros: λ=1.0, α=1.0, maxIter=50

Train-Validation Split:
  - Tiempo: 6.66s
  - RMSE Test: $2,342,402,953.27
  - Hiperparámetros: λ=1.0, α=1.0, maxIter=50

Diferencia RMSE: 0.0


En este experimento se compararon dos estrategias de optimización de hiperparámetros:

1. **Grid Search + Cross-Validation (K=3)**
2. **Train-Validation Split (80/20)**

**Resultados obtenidos:**

- Grid Search + CV:
  - Tiempo: 18.63 segundos
  - RMSE Test: $2,342,402,953.27
  - Hiperparámetros óptimos: λ=1.0, α=1.0, maxIter=50

- Train-Validation Split:
  - Tiempo: 6.66 segundos
  - RMSE Test: $2,342,402,953.27
  - Hiperparámetros óptimos: λ=1.0, α=1.0, maxIter=50

**¿Cuándo usar cada estrategia?**

- Usaría Grid Search + CV cuando:
  - El dataset es pequeño o mediano.
  - El problema es crítico y necesito mayor robustez.
  - Hay alta variabilidad en los resultados.

- Usaría Train-Validation Split cuando:
  - El dataset es grande.
  - Necesito rapidez.
  - El grid es amplio y el costo computacional es alto.

En entornos productivos con Big Data, Train-Validation Split suele ser la opción más eficiente, mientras que Cross-Validation
es preferible en fases exploratorias o académicas.







### **Seleccionar y guardar modelo final con hiperparametros**

In [10]:
import json

# Seleccionar mejor modelo global
mejor_modelo = best_grid_model if rmse_grid <= rmse_tvs else best_tvs_model
estrategia_usada = "Grid Search + CV" if rmse_grid <= rmse_tvs else "Train-Validation Split"
rmse_final = rmse_grid if rmse_grid <= rmse_tvs else rmse_tvs

print(f"Estrategia seleccionada: {estrategia_usada}")
print(f"RMSE final seleccionado: ${rmse_final:,.2f}")

# Guardar modelo
model_path = "/opt/spark-data/raw/tuned_model"
mejor_modelo.write().overwrite().save(model_path)

print(f"✓ Modelo final guardado en: {model_path}")

# hiperparametros optimos
hiperparametros_optimos = {
    "regParam": float(mejor_modelo.getRegParam()),
    "elasticNetParam": float(mejor_modelo.getElasticNetParam()),
    "maxIter": int(mejor_modelo.getMaxIter()),
    "rmse_test": float(rmse_final),
    "estrategia": estrategia_usada
}

json_path = "/opt/spark-data/raw/hiperparametros_optimos.json"

with open(json_path, "w") as f:
    json.dump(hiperparametros_optimos, f, indent=4)

print(f"✓ Hiperparámetros óptimos guardados en: {json_path}")



Estrategia seleccionada: Grid Search + CV
RMSE final seleccionado: $2,342,402,953.27
✓ Modelo final guardado en: /opt/spark-data/raw/tuned_model
✓ Hiperparámetros óptimos guardados en: /opt/spark-data/raw/hiperparametros_optimos.json


### **Refinar grid alrededor de mejores valores**

In [11]:
# Definir grid fino
fine_grid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.7, 0.9, 1.0, 1.1, 1.3]) \
    .addGrid(lr.elasticNetParam, [0.8, 0.9, 1.0]) \
    .addGrid(lr.maxIter, [50, 75]) \
    .build()

print(f"Combinaciones grid fino: {len(fine_grid)}")
print(f"Modelos a entrenar con K=3: {len(fine_grid) * 3}")

# CrossValidator con grid fino
cv_fine = CrossValidator(
    estimator=lr,
    estimatorParamMaps=fine_grid,
    evaluator=evaluator,
    numFolds=3,
    seed=42
)

print("\nEntrenando Grid Fino + CV...")
start_time = time.time()
cv_fine_model = cv_fine.fit(train)
fine_time = time.time() - start_time

print(f"✓ Grid fino completado en {fine_time:.2f} segundos")


Combinaciones grid fino: 30
Modelos a entrenar con K=3: 90

Entrenando Grid Fino + CV...
✓ Grid fino completado en 20.88 segundos


In [12]:
# %%
best_fine_model = cv_fine_model.bestModel

preds_fine = best_fine_model.transform(test)
rmse_fine = evaluator.evaluate(preds_fine)

print("\n=== MEJOR MODELO (Grid Fino) ===")
print(f"regParam:        {best_fine_model.getRegParam()}")
print(f"elasticNetParam: {best_fine_model.getElasticNetParam()}")
print(f"maxIter:         {best_fine_model.getMaxIter()}")
print(f"RMSE Test:       ${rmse_fine:,.2f}")


=== MEJOR MODELO (Grid Fino) ===
regParam:        0.9
elasticNetParam: 0.9
maxIter:         50
RMSE Test:       $2,342,402,953.42


## Preguntas de Reflexión

1. **¿Cuándo usarías Grid Search vs Random Search?**

   **Respuesta:**
   
   Usaría **Grid Search** cuando el espacio de hiperparámetros es pequeño o moderado y quiero explorar todas las combinaciones posibles de forma exhaustiva. Es ideal cuando el número de parámetros es reducido y el costo computacional es manejable.
   
   Usaría **Random Search** cuando el espacio de búsqueda es grande o continuo, ya que permite explorar más combinaciones distintas en menos tiempo. Random Search suele ser más eficiente en alta dimensionalidad, ya que no desperdicia recursos evaluando combinaciones poco prometedoras de manera sistemática.

---

2. **¿Por qué Train-Validation Split es más rápido que CV?**

   **Respuesta:**
   
   Train-Validation Split es más rápido porque cada combinación de hiperparámetros se entrena **una sola vez**, utilizando un único split interno (por ejemplo 80/20).  
   
   En cambio, Cross-Validation con K folds entrena cada combinación **K veces**, lo que multiplica el costo computacional por K. Por eso CV es más robusto, pero también más costoso en tiempo y recursos.

---

3. **¿Qué pasa si el grid es demasiado grande?**

   **Respuesta:**
   
   Si el grid es demasiado grande:
   
   - El tiempo de entrenamiento crece exponencialmente.
   - Puede saturar memoria y recursos del clúster.
   - Se vuelve ineficiente explorar combinaciones poco relevantes.
   
   En entornos Big Data, un grid muy amplio puede volver el proceso inviable. Por eso es recomendable:
   - Usar escala logarítmica.
   - Refinar alrededor de las mejores zonas.
   - Reducir dimensionalidad antes del tuning.

---

4. **¿Cómo implementarías Random Search en Spark ML?**
   (Spark ML no tiene Random Search nativo)

   **Respuesta:**
   
   Para implementar Random Search en Spark ML se podría:
   
   1. Definir rangos continuos para los hiperparámetros.
   2. Generar combinaciones aleatorias usando `random` o `numpy`.
   3. Construir manualmente un `ParamGridBuilder` con esas combinaciones aleatorias.
   4. Pasarlas al `CrossValidator` o `TrainValidationSplit`.
   
   Es decir, simular Random Search creando un grid con muestras aleatorias en lugar de combinaciones exhaustivas. Esto permite explorar el espacio de búsqueda de manera más eficiente cuando los hiperparámetros son muchos o continuos.


In [13]:
spark.stop()